In [ ]:
# extraction des données
import xml.etree.ElementTree as ET
import pymongo
import html2markdown

from lxml import etree
from collections import defaultdict

tree = ET.parse("Posts.xml")
root = tree.getroot()

liste_des_reponsesId=[]
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["NougatBD2"]
mycol = mydb["astro2"]


import time
start_time = time.time()

for child in root:
    type_id = child.attrib['PostTypeId']
    
    try:
        rep_id = child.attrib['AcceptedAnswerId']
    except:
        rep_id = 0
    
    if (type_id=='1' and rep_id !=0) : 
        quest = child.attrib['Body']
        question = html2markdown.convert(quest) 
        liste_des_reponsesId.append(rep_id)
        astro_dict = [{'_id':child.attrib['Id'], 'PostTypeId':1, 'AcceptedAnswerId':rep_id, 'question':question}]
        x = mycol.insert(astro_dict)

for index in liste_des_reponsesId : 
    found = False
    for child in root:
        if index == child.attrib['Id'] :
            rep = child.attrib['Body']
            reponse = html2markdown.convert(rep)
            astro_dict = [{'_id':index , 'PostTypeId':2,  'response':reponse}]
            x = mycol.insert(astro_dict)
            found = True
        if found : break
    
end_time = time.time()
print(end_time - start_time)


In [12]:
print(len(liste_des_reponsesId))

0


In [13]:
myquery = { "PostTypeId": 1 }
questions = mycol.find(myquery)

n = 0
for q in questions : 
    print(q)
    n+=1
    if n==10 : break

{'_id': '1', 'PostTypeId': 1, 'AcceptedAnswerId': '879', 'question': 'Suppose I would like to calculate the inclination of a satellite from the __ecliptic__. Would it be possible to do this with an amateur telescope? How would I go about doing so?\n\n---\n\nNote: A good answer should tell what kind of telescope an amateur would need, what measurements they would need to make, then what calculations they would have to perform to get the inclination (or the "instantaneous angular measurement from the ecliptic at the time of measurement").'}
{'_id': '2', 'PostTypeId': 1, 'AcceptedAnswerId': '45', 'question': 'How are exoplanetary atmosphere compositional spectra distinguished from those of the parent star(s), from the composition of the planetary surface or any other factor? Is it actually possible to determine the atmospheric composition precisely using this method?\n\nAre there any specific examples of such an analysis having been performed on an exoplanet?'}
{'_id': '4', 'PostTypeId': 

In [9]:
myquery = { "PostTypeId": 2 }
questions = mycol.find(myquery)

for rep in questions : 
    print(rep)
    break


{'_id': '879', 'PostTypeId': 2, 'response': 'Any telescope can be made to give you the information that you are looking for. The first thing that you will need to know is the location of the ecliptic which varies throughout the year. Or are you looking to find in relation to the the celestial equator?\n\n[https://en.wikipedia.org/wiki/Celestial\\_equator](https://en.wikipedia.org/wiki/Celestial_equator)\n\nEither way, you would start of the same by find the Declination (D) and Right Ascension (R.A) of the object in question. You do not need a telescope to find this, unless you are unable to see the object with the naked eye. If the telescope has a polar mount and is properly set up. You can read the R.A. and D off of the mount. \n\nOtherwise it will require some trigonometry and knowing where you are (at least your latitude)\n\nYou will need to determine the Altitude and Azimuth of the object. Which is really just the direction from North and the angle from horizontal that your telesco

In [10]:
quest = "I've heard that light can't escape from"  # a taper sur discord
question = "^"+ quest

myquery = { "question": { "$regex": question } }
mydoc = mycol.find(myquery)

for x in mydoc:
    print(x['question'])
    ansID = x['AcceptedAnswerId']
    
print(100*'*')
print('la reponse est de id = ',ansID)
print(100*'*')

myquery = { "_id": ansID }
mydoc = mycol.find(myquery)
for x in mydoc:
    reponse = x['response']
    print(reponse)

I've heard that light can't escape from a [black hole](http://en.wikipedia.org/wiki/Black_hole). Can it? If not, why?
****************************************************************************************************
la reponse est de id =  8
****************************************************************************************************
A black hole has an <a href="http://en.wikipedia.org/wiki/Event_horizon" rel="nofollow noreferrer">event horizon</a> which 'marks the point of no return'. So yes, light cannot escape from a black hole.

Why? Well, think of a 'spacetime fabric'. It's the easiest way to understand the physics at work here, in my opinion.

Usually, the fabric would look like this:

<a href="https://i.stack.imgur.com/7HKzL.jpg" rel="nofollow noreferrer">![fabric](https://i.stack.imgur.com/7HKzL.jpg)</a>  
<sub>(source: <a href="http://whyfiles.org/wp-content/uploads/2011/05/gravity_probespacetime.jpg" rel="nofollow noreferrer">whyfiles.org</a>)</sub> 

However, a bla

In [72]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

def tokenize(corpus):
    #Tokenization
    words=[]
    tokens = nltk.word_tokenize(corpus)
    words.append(tokens)
    flat_words = [item for sublist in words for item in sublist]
    stopW = stopwords.words('english')
    stopW.extend(set(string.punctuation))
    tokens_without_stopwords = [x for x in flat_words if x not in stopW]
    return tokens_without_stopwords

myquery = { "PostTypeId": 1 }
questions = mycol.find(myquery)

liste_questions=[]
for q in questions : 
    liste_questions.append(q['question'])
    
def check(liste, words): 
    res = [] 
    for substring in liste: 
        k = [ w.lower() for w in words if w.lower() in substring.lower() ] 
        if (len(k) == len(words) ): 
            res.append(substring)           
    return res[0] 
    
words = tokenize(' to calculate the inclination of a satellite')
print(words)
check(liste_questions, words)



['calculate', 'inclination', 'satellite']


'Suppose I would like to calculate the inclination of a satellite from the __ecliptic__. Would it be possible to do this with an amateur telescope? How would I go about doing so?\n\n---\n\nNote: A good answer should tell what kind of telescope an amateur would need, what measurements they would need to make, then what calculations they would have to perform to get the inclination (or the "instantaneous angular measurement from the ecliptic at the time of measurement").'